<a href="https://colab.research.google.com/github/NicoZenith/Transformers/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
!pip install transformers
from transformers import AutoTokenizer

import torch
import torch.nn as nn
from torch.nn import functional as F
import math

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

response = requests.get(url)
text = response.text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00


In [ ]:
print("length of dataset in characters", len(text))

length of dataset in characters 1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode([0,2,3,4,5]))

[46, 47, 47, 1, 58, 46, 43, 56, 43]

!$&'


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


We here solely import the BERT tokenizer (nothing else)

If we don't want to do it character-wise, but with word tokens (taken from the BERT tokenizer):

In [ ]:
model_name = "bert-base-uncased"  # Replace with the desired pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)

encode = lambda s: tokenizer.convert_tokens_to_ids(tokenizer.tokenize(s))
decode = lambda l: tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(l))

print(encode("hi there I would like to go to the movies"))
print(decode([7632, 2045, 1045, 2052, 2066, 2000, 2175, 2000, 1996, 5691]))

[7632, 2045, 1045, 2052, 2066, 2000, 2175, 2000, 1996, 5691]
hi there i would like to go to the movies


In [ ]:
vocab_size = tokenizer.vocab_size
print(vocab_size)

30522


In [ ]:


# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 250
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 8
n_layer = 6
dropout = 0.2

use_tokens = True
# ------------

torch.manual_seed(1337)


# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

if use_tokens:
    # create a mapping from text to word tokens
    model_name = "bert-base-uncased"  # Replace with the desired pre-trained model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Add a custom rule to handle line breaks
    text = text.replace('\n', '[')
    encode = lambda s: tokenizer.convert_tokens_to_ids(tokenizer.tokenize(s))
    decode = lambda l: tokenizer.convert_tokens_to_string([token.replace('[', '\n') if token == '[' \
                                                           else token for token in tokenizer.convert_ids_to_tokens(l)])
    vocab_size = tokenizer.vocab_size

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Token indices sequence length is longer than the specified maximum sequence length for this model (328719 > 512). Running this sequence through the model will result in indexing errors


In [ ]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class PositionalEncoding(nn.Module):
    "Implement the PE function"
    def __init__(self, n_embd, p_drop, block_size):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=p_drop)

        pe = torch.zeros(block_size, n_embd, device=device)
        position = torch.arange(0, block_size, device=device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, n_embd, 2, device=device) / n_embd * (-math.log(10000.0))  ) # T/2
        pe[:,0::2] = torch.sin(position * div_term)
        pe[:,1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        output = x + nn.Parameter(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(output)


Create the full Transformer model

In [ ]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.position_embedding_sinus = PositionalEncoding(n_embd, p_drop=0.1, block_size=block_size)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        # pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = self.position_embedding_sinus(tok_emb) # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


    def generate_beam(self, idx, max_new_tokens, beam_size):
    # idx is (B, T) array of indices in the current context

        idx_cond = idx[:, -block_size:]
        logits, loss = self(idx_cond)
        # focus only on the last time step
        logits = logits[:,-1,:] # (B, C)
        softmax = nn.Softmax(dim=-1)
        probs = softmax(logits) # (B, C)

        candidates_idx_next = torch.topk(probs, beam_size, dim=1).indices # (B, k)
        # we create k different sequences, that concatenate each candidate
        store_sequences = []
        for i in range(beam_size):
            sequence = torch.cat([idx, candidates_idx_next[:, i].unsqueeze(1)], dim=1)
            sequence = torch.unsqueeze(sequence, 1)  # Add an extra dimension at dim=1
            store_sequences.append(sequence)
        all_idx = torch.cat(store_sequences, dim=1)

        sequence_probs = torch.topk(probs, beam_size, dim=1).values

        for _ in range(1, max_new_tokens):

            all_idx_cond = all_idx[:, :, -block_size:]

            store_topk_probs = []
            store_topk_indices = []

            for i in range(beam_size):
                logits, _ = self(all_idx_cond[:, i, :])
                logits = logits[:,-1,:] # (B, C)
                probs = softmax(logits)
                topk_probs = torch.topk(probs, beam_size, dim=1).values
                topk_indices = torch.topk(probs, beam_size, dim=1).indices
                store_topk_probs.append(topk_probs)
                store_topk_indices.append(topk_indices)

            # compute all probabilities of sequences
            all_topk_probs = torch.cat(store_topk_probs, dim=1) # contains the k**2 probabilities
            all_topk_indices = torch.cat(store_topk_indices, dim=1) # contains the k**2 indices

            sequence_probs = sequence_probs.repeat_interleave(beam_size, dim=1) # repeat the probabilities k times so we multiply with the new probas
            sequence_probs = sequence_probs * all_topk_probs # multiply to obtain probability of sequence of all selected leaves
            store = torch.topk(sequence_probs, beam_size, dim=1).indices
            idx_topk = all_topk_indices[torch.arange(probs.shape[0]).unsqueeze(1), store]
            sequence_probs = torch.topk(sequence_probs, beam_size, dim=1).values
            # we have the top k indices
            store_sequences = []
            for i in range(beam_size):
                index_for_previous_sequence = store[:,i] // probs.shape[1]
                previous_sequence_to_keep = all_idx[torch.arange(probs.shape[0]), index_for_previous_sequence]
                new_sequence = torch.cat([previous_sequence_to_keep, idx_topk[:,i].unsqueeze(1)], dim=1 )
                new_sequence = torch.unsqueeze(new_sequence, 1)
                store_sequences.append(new_sequence)
            all_idx = torch.cat(store_sequences, dim=1)


        # at the end of the loop, return the indices that have the best probability of sequence
        best_sequence = torch.argmax(sequence_probs, dim=1)
        idx = all_idx[torch.arange(probs.shape[0]), best_sequence]

        return idx

In [ ]:



model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate_beam(context, max_new_tokens=500, beam_size=4)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


34.112058 M parameters
step 0: train loss 10.3680, val loss 10.3665
step 250: train loss 5.9482, val loss 6.0866
step 500: train loss 5.6786, val loss 5.8056
step 750: train loss 5.2354, val loss 5.4062
step 1000: train loss 4.9592, val loss 5.1986
step 1250: train loss 4.7710, val loss 5.0507
step 1500: train loss 4.6266, val loss 4.9088
step 1750: train loss 4.5125, val loss 4.8251
step 2000: train loss 4.4267, val loss 4.7827
step 2250: train loss 4.3939, val loss 4.7882
step 2500: train loss 4.2880, val loss 4.6969
step 2750: train loss 4.2292, val loss 4.6748


KeyboardInterrupt: ignored

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate_beam(context, max_new_tokens=300, beam_size=6)[0].tolist()))

[PAD]s d 
 
 king : 
, 
 tis, 
 i : 
 i ' ll be ach of and. 
 
 kingenius : 
 
 king richard 
 : 
 
, tis iii : 
 i ' ll be, 
 
 
 richard : 
 
ke : 
 i,, 
 lord, 
 i, ll notlingch you 
 
 to iling ll 
 a good - morrow, i havesee - 
 to make the world, that you shall not 
 the people, that he is, but i am, as it, i ' ld not. ' ' tis not, sir. ' tis, i am, sir. ' tis 
 the duke, i am a little. ' tis 
 a man of ay ' st : but, my brother? ' s the king 
 i am a king, that you are not, i 
 not, to make him, i am it, and 
 i have a little. ' tis a little. ' tis 
 that ' twas thou? ' t, i have a 
 that is not to be the world, 
 that i have you to be so, 
 to be the people of a little, and 
 that he shall not, to have a 
 house. ' t, ' tis not not a king, and 
 and i have a little. i have it to the king 
 that ' t ' st i have not, ' 
 that he is the people, that he shall not


In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
encode( '\nhello')

[7592]

In [ ]:
a = encode(text[:100])

In [ ]:
decode([100])

'[UNK]'

In [ ]:
text[:1000]

"First Citizen:[Before we proceed any further, hear me speak.[[All:[Speak, speak.[[First Citizen:[You are all resolved rather to die than to famish?[[All:[Resolved. resolved.[[First Citizen:[First, you know Caius Marcius is chief enemy to the people.[[All:[We know't, we know't.[[First Citizen:[Let us kill him, and we'll have corn at our own price.[Is't a verdict?[[All:[No more talking on't; let it be done: away, away![[Second Citizen:[One word, good citizens.[[First Citizen:[We are accounted poor citizens, the patricians good.[What authority surfeits on would relieve us: if they[would yield us but the superfluity, while it were[wholesome, we might guess they relieved us humanely;[but they think we are too dear: the leanness that[afflicts us, the object of our misery, is as an[inventory to particularise their abundance; our[sufferance is a gain to them Let us revenge this with[our pikes, ere we become rakes: for the gods know I[speak this in hunger for bread, not in thirst for revenge.[

In [ ]:
text[:1000]

"First Citizen: [LB] Before we proceed any further, hear me speak. [LB]  [LB] All: [LB] Speak, speak. [LB]  [LB] First Citizen: [LB] You are all resolved rather to die than to famish? [LB]  [LB] All: [LB] Resolved. resolved. [LB]  [LB] First Citizen: [LB] First, you know Caius Marcius is chief enemy to the people. [LB]  [LB] All: [LB] We know't, we know't. [LB]  [LB] First Citizen: [LB] Let us kill him, and we'll have corn at our own price. [LB] Is't a verdict? [LB]  [LB] All: [LB] No more talking on't; let it be done: away, away! [LB]  [LB] Second Citizen: [LB] One word, good citizens. [LB]  [LB] First Citizen: [LB] We are accounted poor citizens, the patricians good. [LB] What authority surfeits on would relieve us: if they [LB] would yield us but the superfluity, while it were [LB] wholesome, we might guess they relieved us humanely; [LB] but they think we are too dear: the leanness that [LB] afflicts us, the object of our misery, is as an [LB] inventory to particularise their abund

In [ ]:
generated_text = "hello,\nhow are you?\nI'm fine, thanks.\n"
generated_text = generated_text.replace('\n', '\n')

print(generated_text)

hello,
how are you?
I'm fine, thanks.

